<a href="https://colab.research.google.com/github/awaiskhan005/DATA-SCIENCE-AND-AI-/blob/main/Franklinn_XGboast_project_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [7]:
# Import Required Libraries
import os
import joblib
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Install the 'ta' library if it's not already installed
!pip install ta

# Install the 'python-binance' library if it's not already installed
!pip install python-binance # This line is added to install the missing library

# Now import from 'ta'
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
from ta.trend import EMAIndicator
from binance.client import Client # This import should now work
from datetime import datetime, timedelta, timezone
import time

In [8]:
import os
import joblib
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
from ta.trend import EMAIndicator
from binance.client import Client
from datetime import datetime, timedelta, timezone
import time


In [9]:
# Import Required Libraries
import os
import joblib
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
from ta.trend import EMAIndicator
from binance.client import Client
from datetime import datetime, timedelta, timezone
import time

# Initialize Binance Client
API_KEY = "8gexdTyvvgmMQCqeFH9Uw4kDUc7xdlQ7a8ZGYxlszMFUD78Wkwe1cNQYNbKPe7c"
API_SECRET = "g39iujh5i29t6GcMoCYlGXkusnfkE9PMpcZVyNNk1XlOIq2eGumGM6fb9za6DOZ"
client = Client(API_KEY, API_SECRET)

# Define Directories and Symbols
model_dir = "models"
os.makedirs(model_dir, exist_ok=True)

symbols = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT"]

# Fetch and Process Data from Binance API
def fetch_candles(symbol, interval, lookback_days):
    start_time = (datetime.now() - timedelta(days=lookback_days)).strftime('%Y-%m-%d %H:%M:%S')
    candles = client.get_historical_klines(symbol, interval, start_time)
    df = pd.DataFrame(candles, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    for col in ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# Calculate Indicators
def calculate_indicators(df):
    bb = BollingerBands(df['close'], window=12, window_dev=2)
    df['bb_low'] = bb.bollinger_lband()
    df['bb_high'] = bb.bollinger_hband()
    df['rsi'] = RSIIndicator(df['close'], window=14).rsi()
    stoch_rsi = (df['rsi'] - df['rsi'].rolling(window=14).min()) / (df['rsi'].rolling(window=14).max() - df['rsi'].rolling(window=14).min())
    df['stoch_rsi'] = stoch_rsi
    df['fast_ema'] = EMAIndicator(close=df['close'], window=5).ema_indicator()
    df['slow_ema'] = EMAIndicator(close=df['close'], window=15).ema_indicator()
    df['sma_30'] = df['close'].rolling(window=30).mean()
    return df

# Train Models for Each Symbol
for symbol in symbols:
    df = fetch_candles(symbol, Client.KLINE_INTERVAL_6HOUR, 500)
    df = calculate_indicators(df)
    df['future_return'] = df['close'].shift(-80) / df['close'] - 1
    df['target'] = (df['future_return'] > 0.6).astype(int)
    df.dropna(inplace=True)

    # Select only numeric features
    features = df.select_dtypes(include=[np.number]).drop(columns=['target', 'future_return'], errors='ignore')
    target = df['target']

    # Split Data into Train/Test
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # Train XGBoost Model
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)

    # Save Model
    model_path = os.path.join(model_dir, f"{symbol}_xgboost_model.pkl")
    joblib.dump(model, model_path)

    # Evaluate Model
    y_pred = model.predict(X_test)
    print(f"Model for {symbol}:")
    print(classification_report(y_test, y_pred))

# Use Trained Models to Predict Probabilities
def predict_with_trained_models():
    results = {}
    for symbol in symbols:
        model_path = os.path.join(model_dir, f"{symbol}_xgboost_model.pkl")
        if os.path.exists(model_path):
            model = joblib.load(model_path)
            df = fetch_candles(symbol, Client.KLINE_INTERVAL_6HOUR, 10)
            df = calculate_indicators(df)
            features = df.select_dtypes(include=[np.number]).iloc[-1:].drop(columns=['target', 'future_return'], errors='ignore')
            probabilities = model.predict_proba(features)
            percentage_chances = probabilities[:, 1] * 100
            results[symbol] = percentage_chances[0]
    return results

results = predict_with_trained_models()
for symbol, prob in results.items():
    print(f"Probability of price increase >60% for {symbol}: {prob:.2f}%")

# Integration with Trading System
def execute_buy_order_with_model(client, symbol, usdt_to_use):
    results = predict_with_trained_models()
    probability = results.get(symbol, 0)
    if probability > 60:
        print(f"Executing buy order for {symbol} with {usdt_to_use} USDT. Probability: {probability:.2f}%")
        # Mocking buy order execution
        print(f"Buy order executed for {symbol}.")
    else:
        print(f"Buy signal not strong enough for {symbol}. Probability: {probability:.2f}%")


Model for BTCUSDT:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       379

    accuracy                           1.00       379
   macro avg       1.00      1.00      1.00       379
weighted avg       1.00      1.00      1.00       379

Model for ETHUSDT:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       379

    accuracy                           1.00       379
   macro avg       1.00      1.00      1.00       379
weighted avg       1.00      1.00      1.00       379

Model for BNBUSDT:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       377
           1       0.00      0.00      0.00         2

    accuracy                           0.99       379
   macro avg       0.50      0.50      0.50       379
weighted avg       0.99      0.99      0.99       379

Model for SOLUSDT:
              precision    recall  f1-score   suppo